In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('fake_news_train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
# Feature set and Target set
X=df.drop('label',axis=1)
y=df['label']


# Textual data preprocessing

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import re

In [5]:
# Handling missing values
df=df.dropna()

In [6]:
# Re-assign of index after droping of missing values
messages=df.copy()
messages.reset_index(inplace=True)

In [7]:
# Removal of stop words, special characters from the textual data
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

# Creation of bag of words

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,ngram_range=(1,3)) # take most frequent 5000 features and with combinations of 1 word, 2 word, 3 word
X=cv.fit_transform(corpus).toarray() # finally convert into array


In [9]:
X.shape

(18285, 5000)

In [10]:
y=messages['label']

In [11]:
# Training and Testing data
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)


In [12]:
cv.get_feature_names()[:20]

['abandon',
 'abc',
 'abc news',
 'abduct',
 'abe',
 'abedin',
 'abl',
 'abort',
 'abroad',
 'absolut',
 'abstain',
 'absurd',
 'abus',
 'abus new',
 'abus new york',
 'academi',
 'accept',
 'access',
 'access pipelin',
 'access pipelin protest']

In [13]:
# Converting into df
count_df = pd.DataFrame(X_train, columns=cv.get_feature_names())

In [14]:
count_df.head()

,abandon,abc,abc news,abduct,abe,abedin,abl,abort,abroad,absolut,...,zero,zika,zika viru,zionist,zone,zone new,zone new york,zoo,zu,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Multinomial naivebayes
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
classifier=GaussianNB()

In [16]:
# Training and testing
classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
score


0.7335542667771334

# Fake news classification using logistic regression model

In [17]:
from sklearn import svm
svm=svm.SVC(kernel='linear') # Linear kernel


In [18]:
svm.fit(X_train, y_train)
preds=svm.predict(X_test)


In [19]:
score = metrics.accuracy_score(y_test, preds)
score

0.9274233637116819